# Multiple Views

Create our project

In [3]:
# create project
mkdir workspace2
django-admin startproject myproject workspace2
cd workspace2

This time we create a webapp with multiple views.  We will leave using models until the next tutorial.  
First create a new webapp:

In [4]:
python manage.py startapp multiple_views
tree -I __pycache__ .

.
├── manage.py
├── multiple_views
│   ├── admin.py
│   ├── apps.py
│   ├── __init__.py
│   ├── migrations
│   │   └── __init__.py
│   ├── models.py
│   ├── tests.py
│   └── views.py
└── myproject
    ├── asgi.py
    ├── __init__.py
    ├── settings.py
    ├── urls.py
    └── wsgi.py

4 directories, 13 files


Now register our app in the master settings.py file.  
This file is quite long, but the relevant part is at present:

In [5]:
sed -n '/^INSTALLED_APPS/,/^]/p' myproject/settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]


Update the installed apps:

In [6]:
sed -i -e "/INSTALLED_APPS/ a \    'multiple_views'," myproject/settings.py
sed -n '/^INSTALLED_APPS/,/^]/p' myproject/settings.py

INSTALLED_APPS = [
    'multiple_views',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]


This has registered the webapp called `multiple_views`.  
Let's create several views for our app:

In [7]:
cat << EOF > multiple_views/views.py
from django.http import HttpResponse
from django.shortcuts import render


# Create your views here.
def index(request):
    return HttpResponse("The index")

def view1(request):
    return HttpResponse(f"This is view1")

def view2(request, param1, param2):
    return HttpResponse(f"view2: param1 = {param1}, param2 = {param2}")

def view3(request):
    return HttpResponse(f"This will use a template")
EOF

Now create the local url mappings

In [8]:
cat << EOF > multiple_views/urls.py
from django.urls import path

from . import views

urlpatterns = [
    # firefox 'http://localhost:7000/multiple_views'
    path('', views.index, name='index'),

    #firefox 'http://localhost:7000/multiple_views/view1'
    path('view1/', views.view1, name='view1'),

    #firefox 'http://localhost:7000/multiple_views/view2/abc/def'
    path('view2/<param1>/<param2>/', views.view2, name='view2'),

    #firefox 'http://localhost:7000/multiple_views/view3'
    path('view3/', views.view3, name='view3'),
]
EOF

Link our mappings to the master url file in the `myproject` folder

In [9]:
cat << EOF > myproject/urls.py
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('admin/', admin.site.urls),
    path('multiple_views/', include('multiple_views.urls')),
]
EOF

We should now have our webapp active.  Let's perform migrations and start the server on port 7000 of the localhost.

In [10]:
python manage.py migrate
fuser -k 7000/tcp
python manage.py runserver 7000 &

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

Let's use `firefox` to see the app:

In [12]:
firefox http://localhost:7000/multiple_views
firefox --new-tab http://localhost:7000/multiple_views/view1
firefox --new-tab http://localhost:7000/multiple_views/view2/abc/def

[04/Dec/2024 22:12:01] "GET /multiple_views HTTP/1.1" 301 0
[04/Dec/2024 22:12:01] "GET /multiple_views/ HTTP/1.1" 200 9
firefox http://localhost:7000/multiple_views
1
[04/Dec/2024 22:13:42] "GET /multiple_views/ HTTP/1.1" 200 9
[04/Dec/2024 22:13:43] "GET /multiple_views/view1 HTTP/1.1" 301 0
firefox --new-tab http://localhost:7000/multiple_views/view2/abc/def
[04/Dec/2024 22:13:43] "GET /multiple_views/view1/ HTTP/1.1" 200 13


: 1

These views return a str; this is far too restrictive.  In practice we should use a template to return a complete page. To set up the template we first create a folder for the template:

In [13]:
mkdir -p multiple_views/templates

mkdir -p multiple_views/templates


: 1

Now create the template

In [14]:
cat << EOF > multiple_views/templates/mytemplate.html
mytemplate.html<br>
First Name: {{ first_name }} <br>
Last Name: {{ last_name }} <br>
EOF

cat << EOF > multiple_views/templates/mytemplate.html
mytemplate.html<br>
First Name: {{ first_name }} <br>
Last Name: {{ last_name }} <br>
EOF


: 1

This creates the structure:

In [15]:
tree -I __pycache__ multiple_views

tree -I __pycache__ multiple_views
multiple_views
├── admin.py
├── apps.py
├── __init__.py
├── migrations
│   └── __init__.py
├── models.py
├── templates
│   └── mytemplate.html
├── tests.py
├── urls.py
└── views.py

3 directories, 9 files


: 1

We can rewrite `view3` to use the template:

In [16]:
cat << EOF >> multiple_views/views.py
def view3(request):
    context = {
        'first_name':'Chris', 
        'last_name': 'Seddon'
    }
    return render(request, "mytemplate.html", context)
EOF

cat << EOF >> multiple_views/views.py
def view3(request):
    context = {
        'first_name':'Chris', 
        'last_name': 'Seddon'
    }
    return render(request, "mytemplate.html", context)
EOF


: 1

Now view the template in the browser

In [17]:
firefox http://localhost:7000/multiple_views/view3

/home/chris/workspace/python-course/src/38 Django/workspace2/multiple_views/views.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
December 04, 2024 - 22:14:20
Django version 5.1.3, using settings 'myproject.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

firefox http://localhost:7000/multiple_views/view3^Jfirefox http://localhost:7000/multiple_views/view3


: 1

clean up:
* kill the server
* remove workspace

In [18]:
cd ..
fuser -k 7000/tcp
rm -r workspace2

[04/Dec/2024 22:14:25] "GET /multiple_views/view3 HTTP/1.1" 301 0
[04/Dec/2024 22:14:25] "GET /multiple_views/view3/ HTTP/1.1" 200 66
cd ..
fuser -k 7000/tcp
7000/tcp:            52569
rm -r workspace2


: 1